In [12]:
#!pip install --user -r requirements.txt

In [30]:
import sys
import os
# Set up TWINT config
import twint
# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio

# stopword
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw_nltk = stopwords.words('english')

from keras.models import Sequential
from keras.layers import LSTM,Flatten,Embedding,Dense,Conv2D,MaxPooling2D

[nltk_data] Downloading package stopwords to /Users/swu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
nest_asyncio.apply()
c = twint.Config()
c.Search = "BTC"
c.Lang = "en"
# c.Username = "noneprivacy"
# c.Custom["tweet"] = ["id"]
# c.Custom["user"] = ["bio"]

c.Store_pandas = True
# c.Store_csv = True
# c.Output = "test.csv"
c.Limit = 100
c.Pandas = True
twint.run.Search(c)

1491862711176765446 2022-02-10 11:52:42 -0800 <coinprensi2021> @PitmagCommunity @BscPitbull_TR @RoboWolf8 @Crypt0mummy @bitgetglobal @swapzoneio @BitcoinMagazine @KenneltoCouch @BartBaker Don't be late buy more $pit - #ForzaPitbull BscPitbull Our target is 1 billion market value by the end of the year. #PitbullCREW #PitbullTokenShibaKiller #Metaverse #P2E #NFT #Bsc #Bnb #BTC  https://t.co/EVWi7fGZvJ
1491862705657065472 2022-02-10 11:52:41 -0800 <Twiltydo> @RaduDorin9 @BTC_Archive Most people are on bitcoin just for profit. In the end everyone has switched back to their fiat currency. It has become more a asset then a currency.
1491862705375957005 2022-02-10 11:52:41 -0800 <DrDr62989046> the strongest bullish signal flashed for #Bitcoin. Not trend reverse but price bounce is anticipated!  @Tradeosiann is a super #cryptocurrency Trader  I’ve been following his tweets and tips seriously &amp; I’ve been doing great #Metaverse #BTC #DOGE #NFT(&amp;$#
1491862703555633156 2022-02-10 11:52:40 

1491862494444498950 2022-02-10 11:51:50 -0800 <ChandlerRuiz4> &lt;&lt;Today's Pizza in Tomorrow's Bitcoin&gt;&gt;🍕🍕🍕 How much did your pizza cost in future bitcoin #NFT #NFTs #nftart #BTC  https://t.co/3aNUsJIFQw
1491862488647876612 2022-02-10 11:51:49 -0800 <DrDr62989046> the strongest bullish signal flashed for #Bitcoin. Not trend reverse but price bounce is anticipated!  @Tradeosiann is a super #cryptocurrency Trader  I’ve been following his tweets and tips seriously &amp; I’ve been doing great #Metaverse #BTC #DOGE #NFT&amp;-#
1491862486831804418 2022-02-10 11:51:48 -0800 <Steef_Kas> @CryptoIRELAND1 Aaaaaaand thats how the $ gets killed.  Imagine that you just can print #BTC it would be worthless instantly.
1491862486361927706 2022-02-10 11:51:48 -0800 <santhoshvizag> $BTC would be nice if computer algos didn't trade this in lock step with equities and the Fed Reserve would STFU 👍
1491862483727966213 2022-02-10 11:51:48 -0800 <coincapitan> $BTC Price: $44719 $ETH Price: $3140 $LTC 

1491862305650483208 2022-02-10 11:51:05 -0800 <Yusuf1Aksu> @yseminaksoy Btc 393xx
1491862305600008283 2022-02-10 11:51:05 -0800 <JDxBruce> FollowBack 🏇 #Bitcoin #BTC #Cryptos ifb Airdrop @cctip_com draw 1200 SHIB 100  👉 Shill me your Gem 💎 🚀   https://t.co/btz1jImr9m
1491862304526266376 2022-02-10 11:51:05 -0800 <CryptoBazz_> #Crypto Bears talking about resistance rejection and how we’ll now resume the Bear Market 😂😂  Can’t wait what they come up with at $50k, $55k, $60k, $65k, New ATHs. 😂😂  $BTC $XRP $ETH $LTC $QNT
1491862289049288704 2022-02-10 11:51:01 -0800 <BloodFaceBear> TA scale: Right 100% =impossible / God Right 90% = impossible / Batman Right 80% =possible / Sheldon Right 70% =possible / Wolf Right 60% =possible / Wallstreet Moral = you'll nevr find the "1" Find opposing TA's &amp; respect their findings &amp; draw your own conclusions $vra #BTC #ETH
1491862288583716873 2022-02-10 11:51:01 -0800 <crypto910bot> 直近1分間で大きな価格変動がありました BTC-MOVE-0210    -10.14%
1491862287627563012 2

In [6]:
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]
df = twint_to_pandas(["date", "username","tweet", "hashtags", "nreplies", "nretweets","nlikes"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       111 non-null    object
 1   username   111 non-null    object
 2   tweet      111 non-null    object
 3   hashtags   111 non-null    object
 4   nreplies   111 non-null    int64 
 5   nretweets  111 non-null    int64 
 6   nlikes     111 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 6.2+ KB


In [7]:
df.head(5)

,date,username,tweet,hashtags,nreplies,nretweets,nlikes
0,2022-02-10 10:33:20,blackshamrocks1,@tedtalksmacro Ted! May we kindly have an upda...,[],0,0,0
1,2022-02-10 10:33:20,mdsiam15849676,"Trending on #LunarCrush: ""Inside the Chess Ma...",[lunarcrush],0,0,0
2,2022-02-10 10:33:19,Zeze_Kitapsever,@btc_magazin 🙏🙏🙏👏👏,[],0,0,0
3,2022-02-10 10:33:18,MccryptA,Another WINNER!! Be the next lucky one to win ...,[],0,0,0
4,2022-02-10 10:33:17,kripto_teknik,Canlı yayınımız başladı. Bitcoin ve altcoinler...,"[bitcoin, kriptopara]",1,0,0


## Sentiment analysis

### 1. Preprocessing

In [13]:
def clean_anonymization(tweet):
    res=[]
    # Lowercase
    tweet = tweet.lower()
    # Remove single letter words
    # tweet = ' '.join( [w for w in tweet.split() if len(w)>1] )
    # Remove stopword
    tweet = ' '.join([word for word in tweet.split() if word.lower() not in sw_nltk])
    # remove the word after @ OR #
    for i in tweet.split():
        if i.startswith("@") or i.startswith("#"):
            continue
        else:
            res.append(i)
    return ' '.join(res)

In [14]:
df['tweet']=df['tweet'].apply(lambda x:clean_anonymization(x))

<ipython-input-14-79f1e2b532d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet']=df['tweet'].apply(lambda x:clean_anonymization(x))


In [15]:
df.head()

,date,username,tweet,hashtags,nreplies,nretweets,nlikes
0,2022-02-10 10:33:20,blackshamrocks1,ted! may kindly update today's events outlook ...,[],0,0,0
1,2022-02-10 10:33:20,mdsiam15849676,"trending ""inside chess match led feds $3.6 bil...",[lunarcrush],0,0,0
2,2022-02-10 10:33:19,Zeze_Kitapsever,🙏🙏🙏👏👏,[],0,0,0
3,2022-02-10 10:33:18,MccryptA,another winner!! next lucky one win $50 $btc. ...,[],0,0,0
4,2022-02-10 10:33:17,kripto_teknik,canlı yayınımız başladı. bitcoin altcoinlerdek...,"[bitcoin, kriptopara]",1,0,0


### 2. Stacked-LSTM 

First model has embedding layer, convolution layer, max-pooling layer, LSTM layer and classification layer. CNN layer improves the processing time but operation can lose information on local order of words, so accurate sequential tagging is difficult to be analyzed in
convolution layer. Therefore, LSTM layer has been added to process the sequential data [13].

The second model has multi-layered LSTM layers instead of convolution and max-pooling layer. When a LSTM layer processes a single input sequence of time-series data, each cell returns single value of whole sequence in two-dimensional array. Stacked-LSTM (S-LSTM) model allows greater model complexity and learn a hierarchical representation of time-series data. Time-series data are split into 𝑛 number of time periods for training the proposed model. The overview of sentiment analysis and polarity identification using proposed model is depicted in Fig. 3. 𝐷𝑛 represents the oldest DL model that is trained with oldest knowledge base and 𝐷1 is the most current DL model that has been trained with current review data.

In [29]:
# i set 10 as the vocabulary size,as i will be encoding numbers 0 to 9
# I want the length of the word vector to be 4,hence output_dim is set to 4. 
# The length of the input sequence to embedding layer will be 2.

model = Sequential()
# adding an embedding layer
embedding_layer = Embedding(input_dim=10, #Size of the vocabulary
                            output_dim=4, #Length of the vector for each word
                            input_length=2 # Maximum length of a sequence
                           )

# adding convolution layer
lstm = LSTM(176, dropout=0.2, recurrent_dropout=0.2)
# add max pooling layer
Conv2D_layer = Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu")
max_pooling_layer=MaxPooling2D(pool_size=2)
# adding an LSTM layer
lstm = LSTM(176, dropout=0.2, recurrent_dropout=0.2)

model.add(embedding_layer)
model.add(lstm)
model.add(Conv2D_layer)
model.add(max_pooling_layer)
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
print(model.summary())

ValueError: Input 0 of layer "conv2d_2" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 176)

In [ ]:
# train the model for 100 epochs
model.fit(padded_reviews,labels,epochs=100,verbose=0)

In [13]:
from textblob import TextBlob
from IPython.display import Markdown, display

In [14]:
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [15]:
for tweet in df['tweet']:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)
    if analysis.sentiment[0]>0:
        printmd('Positive', color="green")
    elif analysis.sentiment[0]<0:
        printmd('Negative', color="red")
    else:
        printmd("Neutral", color="grey")
        print("")

@Israelm16432 @Tarjas5 @Cryptobullza @BTC_Archive So this happened to me for 3-weeks and I tried so many means but all failed, luckily for me I got in contact with this Hacker pro.craxk on Instagram, he was the one who helped me, he should help you with yours.
Sentiment(polarity=0.1111111111111111, subjectivity=0.5444444444444444)


<span style='color:green'>Positive</span>

@Barbarasllv6 Boa noite 🔥 chegou com fogo na btc né
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>Neutral</span>


I had a dream that #BTC reached $ 79,000🤔
Sentiment(polarity=0.0, subjectivity=0.0)


...

<span style='color:green'>Positive</span>

Tryna fly bae overseas but we up &amp; down like BTC
Sentiment(polarity=0.32222222222222224, subjectivity=0.5944444444444444)


<span style='color:green'>Positive</span>

@saylor #BTC Bossssss 🔥💀🌹🙌💎
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>Neutral</span>

In [16]:
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
# Function to get sentiment 
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0 # Neutral
    elif temp >= 0.0:
        return 1.0 # Positive
    else:
        return 2.0 # Negative
# UDF to write sentiment on DF
sentiment = udf(apply_blob, DoubleType())
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]
def tweets_sentiment(search, limit=1):
    c.Search = search
    # Custom output format
    c.Format = "Username: {username} |  Tweet: {tweet}"
    c.Limit = limit
    c.Pandas = True
    with HiddenPrints():
        print(twint.run.Search(c))
    
    # Transform tweets to pandas DF
    df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])
    
    # Transform Pandas DF to Optimus/Spark DF
    df = op.create.data_frame(pdf= df_pd)
    
    # Clean tweets
    clean_tweets = df.cols.remove_accents("tweet") \
                 .cols.remove_special_chars("tweet")
    
    # Add sentiment to final DF
    return clean_tweets.withColumn("sentiment",    sentiment(clean_tweets['tweet']))

## Add sentiments to dataframe